In [1]:
import os
import time
import sys
import json
import subprocess
import numpy as np
from subprocess import PIPE
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import pickle

# !pip3 install networkx

In [3]:
# define configs
# compiler = NC4HLS.Compiler()
# compiler.model_file = "./test_model/keras_mnist_DCAE.h5"
# compiler.weight_file = "./test_model/keras_mnist_DCAE_weights.h5"
# compiler.export_pynq_code_path = "./pynq/"
# compiler.export_hls_code_path = "./hls/"
export_cpu_code_path = "./cpu/"
# compiler.log_path = "./log/"
# compiler.src_code_path = "../src"
# compiler.thread = 7
# compiler.show_graph = False
# compiler.print_debug = True
model_file = "./test_model/keras_mnist_DCAE.h5"
weight_file = "./test_model/keras_mnist_DCAE_weights.h5"

json_file = export_cpu_code_path + "Config.json"
with open(json_file, "r") as jf:
    ip_config = json.load(jf)

data_types_dict = ip_config["data_types"]
thread_num = ip_config["thread_num"]

data_types_dataset = np.float32
tf.keras.backend.set_floatx('float32')
data_types = data_types_dict

# generate a SharedObject
# command = "cd " + export_cpu_code_path + " && ./GenerateSharedObject.sh\n cd ./.."
# proc = subprocess.run(command, shell=True, stdout=PIPE, stderr=PIPE, text=True)
# print("stdout : {}".format(proc.stdout))
# print("stderr : {}".format(proc.stderr))

src_size, dst_size = (1, 28, 28, 1), (1, 28, 28, 1)

test_model = keras.models.load_model(model_file)
test_model.load_weights(weight_file)

2022-01-11 11:40:47.169646: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [10]:
src_data_name = ["./test_img/src_data_{:03}.png".format(i) for i in range(100)]
execute_time = []

for cnt, src_img in enumerate(src_data_name):
    _execute_time = {}
    _ex_time = {}
    # fake_data_src = np.array(Image.open('./../test_img/keras_mnist_DCAE_input_2D.png'), dtype=data_types_dataset)
    fake_data_src = np.array(Image.open(src_img), dtype=data_types_dataset)
    fake_data_src = (fake_data_src - np.min(fake_data_src)) / (np.max(fake_data_src) - np.min(fake_data_src))
    fake_data_src = [fake_data_src.reshape(src_size)]

    print("\n\n\n----Run Keras Model---\n\n\n")
    start = time.perf_counter()
    ipunit_output = test_model.predict(fake_data_src)
    end = time.perf_counter()
    _ex_time["run core"] = (end - start) * 1000  # [ms]
    _execute_time[0] = _ex_time
    # run the IPUnit
    # print("\n\n\n----Run IPUnit---\n\n\n")

    # data_vec_config = ip_config["data_vec_config"]
    # _src_data = [[0 for x in range(y)] for y in data_vec_config.values()]
    # for bank, i in enumerate(fake_data_src):
    #     _i = i.reshape(-1).tolist()
    #     for j, k in enumerate(_i):
    #         _src_data[bank][j] = k
    # opcode = ip_config["opcode"]

    # control_code_file = export_cpu_code_path + "control_code.bin"
    # weights_vec_file = export_cpu_code_path + "Weights.bin"
    # shared_object = export_cpu_code_path + "CoreUnit_CPU.so"
    # ipunit_output, _execute_time = IPUnit_CPU.IPUnit_CPU_execute_time(_src_data,
    #                                                                  opcode,
    #                                                                  control_code_file=control_code_file,
    #                                                                  weights_vec_file=weights_vec_file,
    #                                                                  shared_object=shared_object,
    #                                                                  data_types=data_types_dict,
    #                                                                  print_debug=False)
    execute_time.append(_execute_time)




----Run Keras Model---





2022-01-11 11:46:55.437069: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-11 11:46:55.438933: W tensorflow/core/platform/profile_utils/cpu_utils.cc:116] Failed to find bogomips or clock in /proc/cpuinfo; cannot determine CPU frequency





----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Keras Model---






----Run Ker

In [11]:
run_time = []
for i, single_record in enumerate(execute_time):
    for j, kv in enumerate(single_record.items()):
        k, v = kv
        if j == 0:
            fields = list(v.keys())
            single_run_time = [0.0 for x in range(len(v))]
        _values = list(v.values())
        print(_values)
        for k, _time in enumerate(_values):
            single_run_time[k] += _time
    run_time.append(single_run_time)

print()
print(fields)
print(run_time)
print("total_time(average) : {}[ms]".format(sum(map(sum, run_time)) / len(run_time)))
with open("./result_img/result_time_rpi_keras.time", "wb") as f:
    pickle.dump({"field": fields, "run_time": run_time, "execute_time": execute_time, "device": "rpi4_4gb_keras"}, f)

[492.1905999071896]
[94.04306299984455]
[94.05617509037256]
[96.67110117152333]
[103.23013225570321]
[99.70495104789734]
[100.445959251374]
[94.51909782364964]
[96.73864115029573]
[94.4406520575285]
[93.64404901862144]
[99.0773350931704]
[94.23519996926188]
[113.5286851786077]
[95.22677166387439]
[93.48491299897432]
[97.17776719480753]
[97.22960321232677]
[95.65280610695481]
[98.1374871917069]
[93.75092294067144]
[94.27647897973657]
[115.63229793682694]
[94.39601982012391]
[93.80379505455494]
[97.32542233541608]
[93.68064207956195]
[94.26271868869662]
[98.50455494597554]
[96.9374249689281]
[111.77158588543534]
[95.49293015152216]
[93.93167030066252]
[96.70451004058123]
[93.48756168037653]
[94.32470332831144]
[97.48414903879166]
[94.56465486437082]
[94.59519386291504]
[116.13177787512541]
[97.74130582809448]
[95.83353390917182]
[96.22577065601945]
[93.75062584877014]
[93.47856091335416]
[96.66595282033086]
[93.72658934444189]
[112.26452933624387]
[95.35097982734442]
[93.86624209582806]


In [12]:

def export_as_png(data: list, shape, file_name: str):
    _output = np.array(data).reshape(shape)
    _output = (_output - np.min(_output)) / (np.max(_output) - np.min(_output)) * 255
    _output = _output.astype(np.uint8)
    _img = Image.fromarray(_output)
    _img.save(file_name)

_dst_size = dst_size[0] * dst_size[1] * dst_size[2] * dst_size[3]
# ipunit_output[0] = ipunit_output[0][0:_dst_size]

export_as_png(ipunit_output, dst_size[1:3], "./result_img/ipunit_output_rpi_keras.png")
Image.open("./result_img/ipunit_output_rpi_keras.png")

In [13]:
output_bin_files = ["./result_img/rpi_keras_output.bin"]
output_bin_data = [ipunit_output]
for serialized_item in zip(output_bin_files, output_bin_data):
    with open(serialized_item[0], "wb") as f:
        pickle.dump(serialized_item[1], f)